In [2]:
%pylab inline

from __future__ import print_function
from __future__ import division

import sklearn
import numpy
import scipy
import pandas
import matplotlib
import seaborn

import json
import re

from matplotlib import pyplot
from collections import Counter

Populating the interactive namespace from numpy and matplotlib


In [3]:
rcParams['figure.figsize'] = 15, 8

Los mails se componen de varios campos entre los que está el emisor, el destinatario y el mensaje. Primero, procesaremos todos los mensajes para identificar cada uno de esos campos y su correspondiente valor.

Cosas que quiero testear si se cumplen para todos los mensajes
--------------------------------------------------------------

- todos los campos tienen un nombre excepto el mensaje
- separador de campos: '\r\n'
- separador del mensaje del resto de los campos: 2*sep_campos
- separador entre nombre de campo y campo: ':'

Preguntas
---------

- el mensaje siempre es el último campo ?
- Los mensajes están mezclados? Puedo usar el tail como test?


Atributos posibles
------------------

- qué están atributos presentes
- orden de los atributos
- largo del mensaje
- propiedades sobre ciertos atributos:
    - emisor
        - dominio de la dirección
        - largo de la dirección
    - destinatario
        - cantidad direcciones
    - asunto
        - largo
        - análisis parecido al mensaje: palabras presentes, etc.
- si hay manera de identificar si el mail fue forwardeado creo que sería bueno, casi nadie debe forwardea spam


In [4]:
linesep = '\r\n'
bodysep = linesep * 2 

def valid(msg):
    return bodysep in msg

def parse2(messages):
    return [
        {
            x.split(':')[0]: x.split(':')[1]
            for x in k.split(bodysep)[0].split(linesep) + [u'message:{}'.format(k.split(bodysep)[1])]
            if ':' in x
        }
        for k in [m.replace('\n\r\t', '') for m in messages]
    ]

def parse(messages):
    parsed = []
    for k in messages:
        for x in k.split(linesep):
            parsed.append((x.split(':')[0], x.split(':')[1]) if ':' in x else x)
                
    return parsed

In [6]:
ham_json = json.load(open('data/ham_train.json'))
ham = parse(ham_json)
ham[:20]

NameError: name 'parse' is not defined

In [7]:
spam_json = json.load(open('data/spam_train.json'))
spam = parse(x for x in spam_json if valid(x))
spam[:20]

ValueError: No JSON object could be decoded

In [ ]:
def key_barplot(p, limit = 0):
    from operator import itemgetter
    keys = __builtin__.sum((Counter(x.keys()) for x in p), Counter())
    vals = sorted(((x, y) for x, y in keys.iteritems() if y > limit), key = itemgetter(1))
    pyplot.xticks(rotation = 90)
    seaborn.barplot(map(itemgetter(0), vals), map(itemgetter(1), vals))

In [ ]:
key_barplot(ham, limit = 5)

In [ ]:
key_barplot(spam, limit = 0)

# Testing

In [43]:
import re

In [14]:
def parse(t):
    headers, msg = 
    return [
        tuple(x.split(':', 1)) if re.match(r'\S*:', x) else ('message', x)
        # tuple(x.split(':', 1))
        for x in t.split('\r\n')
    ]

In [7]:
ham_json = json.load(open('data/ham_train.json'))

In [15]:
ham = map(parse, ham_json)

In [18]:
print(ham_json[1])

message-id: <2011334.1075840007235.javamail.evans#####>
date: wed, 10 oct 2001 16:24:31 -0700 (pdt)
from: kysa.alport#####
to: shift.dl-portland#####
subject: real time deals for october 11th
mime-version: 1.0
content-type: text/plain; charset=us-ascii
content-transfer-encoding: 7bit
x-from: alport, kysa </o=#####/ou=na/cn=recipients/cn=kalport>
x-to: dl-portland real time shift </o=#####/ou=na/cn=recipients/cn=dl-portlandrealtimeshift>
x-cc: 
x-bcc: 
x-origin: williams-w3
x-filename: 

 	  	  	  	  	  	  	  	  	  	  	  	  	  	 
desk	 short term northwest	 short term california	 short term southwest	 short term west bom	  	 
flow date	 10/11/2001	 10/11/2001	 10/11/2001	 10/11/2001	 10/11/2001	 10/11/2001	 10/11/2001	 10/11/2001	 10/11/2001	 10/11/2001	 10/11/2001	 10/11/2001	  	 
deal number	 812532	 812531	  	 nothing to real time	 nothing to real time	 812559	  	  	  	 
delivery point	 mid c	 mid c	  	  	  	  	  	  	  	 sp-15	  	  	  	 
ctpy	 pge	 pge	  	  	  	  	  	  	  	 ca imbala

In [16]:
ham[1]

[(u'message-id', u' <2011334.1075840007235.javamail.evans#####>'),
 (u'date', u' wed, 10 oct 2001 16:24:31 -0700 (pdt)'),
 (u'from', u' kysa.alport#####'),
 (u'to', u' shift.dl-portland#####'),
 (u'subject', u' real time deals for october 11th'),
 (u'mime-version', u' 1.0'),
 (u'content-type', u' text/plain; charset=us-ascii'),
 (u'content-transfer-encoding', u' 7bit'),
 (u'x-from', u' alport, kysa </o=#####/ou=na/cn=recipients/cn=kalport>'),
 (u'x-to',
  u' dl-portland real time shift </o=#####/ou=na/cn=recipients/cn=dl-portlandrealtimeshift>'),
 (u'x-cc', u' '),
 (u'x-bcc', u' '),
 (u'x-origin', u' williams-w3'),
 (u'x-filename', u' '),
 ('message', u''),
 ('message',
  u' \t  \t  \t  \t  \t  \t  \t  \t  \t  \t  \t  \t  \t  \t \ndesk\t short term northwest\t short term california\t short term southwest\t short term west bom\t  \t \nflow date\t 10/11/2001\t 10/11/2001\t 10/11/2001\t 10/11/2001\t 10/11/2001\t 10/11/2001\t 10/11/2001\t 10/11/2001\t 10/11/2001\t 10/11/2001\t 10/11/2001\t

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

&nbsp;

